In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sqlalchemy import text

Inserting data base information and connecting to it

In [2]:

URI = "database-2.ckmnj1f5m6qh.eu-west-1.rds.amazonaws.com"
PORT = "3306"
DB = "backupdata"
USER = "admin"
PASSWORD = "DublinBikes3"

engine = create_engine("mysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

Want to get features to compare in train_test split, I will use datetime to try predict availability first

In [3]:
# Creating get_features function
def get_features():
    # Get datetime and station number for data
    features_raw = engine.execute("SELECT datetime, number FROM availability2;")

    # Create list to store station numbers, days and hours in.
    numbers_days_hours = []
    for row in features_raw:
        
        # Will store each trio in
        number_day_hour = []
        
        # Split epoch time into day and hour
        epoch_time = row[0]
        day = (floor(epoch_time / 86400) + 4) % 7
        number_day_hour.append(row[1])
        number_day_hour.append(day)
        number_day_hour.append(datetime.datetime.fromtimestamp(row[0]).hour)
        numbers_days_hours.append(number_day_hour)
    return numbers_days_hours

def get_availability():
    availability_raw = engine.execute("SELECT available_bikes FROM availability2;")
    # This returns the number of available bikes
    availability = []
    for row in availability_raw:
        availability.append(row[0])
    return availability
                                     

In [4]:

X = get_features()
y = get_availability()

X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

2023-04-06 12:36:29,178 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-06 12:36:29,180 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-06 12:36:29,198 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-06 12:36:29,199 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-06 12:36:29,206 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-06 12:36:29,207 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-06 12:36:29,224 INFO sqlalchemy.engine.Engine SELECT datetime, number FROM availability2;
2023-04-06 12:36:29,225 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-06 12:36:36,047 INFO sqlalchemy.engine.Engine SELECT available_bikes FROM availability2;
2023-04-06 12:36:36,047 INFO sqlalchemy.engine.Engine [raw sql] ()


[[54, 6, 21],
 [98, 0, 2],
 [97, 2, 9],
 [25, 4, 23],
 [84, 2, 0],
 [103, 0, 18],
 [26, 5, 23],
 [103, 0, 21],
 [101, 4, 9],
 [26, 4, 0],
 [24, 2, 10],
 [66, 3, 0],
 [107, 5, 6],
 [96, 6, 1],
 [20, 2, 18],
 [19, 0, 0],
 [111, 5, 19],
 [108, 4, 3],
 [74, 3, 3],
 [67, 0, 1],
 [44, 1, 23],
 [7, 2, 19],
 [21, 0, 13],
 [63, 3, 12],
 [36, 2, 7],
 [85, 4, 15],
 [16, 6, 7],
 [96, 2, 20],
 [23, 5, 4],
 [5, 4, 0],
 [55, 1, 17],
 [6, 6, 10],
 [53, 2, 19],
 [78, 6, 20],
 [109, 5, 14],
 [75, 5, 2],
 [75, 6, 15],
 [33, 0, 20],
 [110, 0, 4],
 [47, 1, 16],
 [89, 0, 4],
 [59, 0, 20],
 [74, 6, 23],
 [17, 0, 8],
 [47, 0, 14],
 [9, 5, 4],
 [47, 5, 8],
 [71, 3, 2],
 [101, 1, 18],
 [31, 2, 17],
 [53, 6, 6],
 [7, 0, 18],
 [33, 4, 11],
 [105, 6, 22],
 [75, 0, 8],
 [24, 3, 5],
 [8, 2, 15],
 [75, 1, 1],
 [56, 2, 5],
 [42, 4, 22],
 [101, 1, 19],
 [66, 6, 18],
 [92, 0, 13],
 [29, 0, 3],
 [20, 2, 6],
 [72, 1, 23],
 [63, 3, 23],
 [10, 6, 14],
 [112, 3, 3],
 [16, 1, 4],
 [68, 3, 3],
 [59, 3, 15],
 [89, 1, 10],
 [94,

In [5]:

# fit final model
model = SGDRegressor() # choose your own model here...
model.fit(X_train, y_train)

Xnew = X_test # query features

# make a prediction
ynew = model.predict(X_test)
for i in range(10):
    print("X=%s, Predicted=%s, Actual = %s" % (Xnew[i], ynew[i], y_test[i]))

X=[110, 6, 1], Predicted=8.92790012937294, Actual = 11
X=[30, 6, 9], Predicted=11.598154286804073, Actual = 0
X=[80, 5, 9], Predicted=10.172274904712168, Actual = 9
X=[56, 3, 22], Predicted=11.531108213459746, Actual = 2
X=[22, 3, 21], Predicted=12.140179951671684, Actual = 14
X=[109, 0, 9], Predicted=7.778574022522489, Actual = 6
X=[110, 4, 20], Predicted=10.489430995675011, Actual = 13
X=[9, 4, 12], Predicted=11.6977333138857, Actual = 12
X=[97, 3, 10], Predicted=9.218577593753398, Actual = 0
X=[33, 3, 23], Predicted=12.143460337742747, Actual = 10


In [6]:
# X_train and y_train are the training data
model = SGDRegressor()
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)
